In [0]:
# MAGIC %md
# MAGIC # 08 - Batch Predictions Pipeline
# MAGIC 
# MAGIC **Generate predictions on new data using the production model**
# MAGIC 
# MAGIC ## Objectives:
# MAGIC - Load production model from MLflow
# MAGIC - Process new/unseen data
# MAGIC - Apply same preprocessing as training
# MAGIC - Generate predictions
# MAGIC - Save results with confidence intervals
# MAGIC - Generate prediction reports

In [0]:
# MAGIC %md
# MAGIC ## 1. Setup & Imports

In [0]:
# Restart Python to ensure clean imports
%restart_python

In [0]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complete")

In [0]:
# MAGIC %md
# MAGIC ## 2. Project Setup

In [0]:
print("="*60)
print("BATCH PREDICTIONS PIPELINE")
print("="*60)

# Define project root
project_root = "/Workspace/COMM - Commercial Analytics (CMAN)/MMM Quattro 2025/Satish/MLFLOW_sample"

# Add to path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"\n📂 Project root: {project_root}")
print(f"✅ Added to sys.path")

# Import custom modules
from src.utils import ConfigLoader, DataLoader, safe_display
from src.feature_engineering import FeatureEngineer

print(f"✅ Custom modules imported")
print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 3. Load Configuration

In [0]:
print("="*60)
print("LOADING CONFIGURATION")
print("="*60)

config_path = f'{project_root}/config/config.yaml'
config = ConfigLoader.load_config(config_path)

print(f"\n✅ Configuration loaded")
print(f"  • Project: {config['project']['name']}")
print(f"  • Model Registry: {config['mlflow']['model_registry_name']}")
print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 4. Setup MLflow & Load Production Model

In [0]:
print("="*60)
print("LOADING PRODUCTION MODEL")
print("="*60)

experiment_name = config['mlflow']['experiment_name']
model_registry_name = config['mlflow']['model_registry_name']

mlflow.set_experiment(experiment_name)
client = MlflowClient()

# Load production model
model_uri = f"models:/{model_registry_name}/Production"

try:
    production_model = mlflow.sklearn.load_model(model_uri)
    
    print(f"\n✅ Production model loaded")
    print(f"  • Model: {model_registry_name}")
    print(f"  • Stage: Production")
    print(f"  • Type: {type(production_model).__name__}")
    
    # Get model version info
    model_versions = client.search_model_versions(f"name='{model_registry_name}'")
    production_version = [v for v in model_versions if v.current_stage == 'Production'][0]
    
    print(f"  • Version: {production_version.version}")
    print(f"  • Created: {datetime.fromtimestamp(production_version.creation_timestamp / 1000).strftime('%Y-%m-%d %H:%M')}")
    
    # Get model metrics
    run = client.get_run(production_version.run_id)
    metrics = run.data.metrics
    
    print(f"\n📊 Model Performance:")
    print(f"  • RMSE: ${metrics.get('test_rmse', 0):,.2f}")
    print(f"  • MAE:  ${metrics.get('test_mae', 0):,.2f}")
    print(f"  • R²:   {metrics.get('test_r2', 0):.4f}")
    
except Exception as e:
    print(f"\n❌ Error loading production model: {e}")
    print(f"  • Make sure a model is in Production stage")
    raise

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 5. Load Preprocessing Objects

In [0]:
print("="*60)
print("LOADING PREPROCESSING OBJECTS")
print("="*60)

processed_path = config['data']['processed_path']

try:
    # Load label encoders
    with open(f"{processed_path}label_encoders.pkl", 'rb') as f:
        label_encoders = pickle.load(f)
    
    print(f"\n✅ Label encoders loaded")
    print(f"  • Categorical features: {len(label_encoders)}")
    
    # Load scaler
    with open(f"{processed_path}scaler.pkl", 'rb') as f:
        scaler = pickle.load(f)
    
    print(f"✅ Scaler loaded")
    print(f"  • Feature count: {len(scaler.feature_names_in_)}")
    
    print(f"\n📋 Expected Features:")
    for i, feature in enumerate(scaler.feature_names_in_, 1):
        print(f"  {i:2d}. {feature}")
    
except Exception as e:
    print(f"\n❌ Error loading preprocessing objects: {e}")
    print(f"  • Make sure you've run the feature engineering notebook")
    raise

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 6. Load New Data for Predictions
# MAGIC 
# MAGIC **Note:** This example uses the test set. In production, you would load new unseen data.

In [0]:
print("="*60)
print("LOADING NEW DATA")
print("="*60)

# Option 1: Load from test set (for demonstration)
# In production, replace this with your new data source

print(f"\n📊 Loading data for predictions...")
print(f"  • Source: Test set (for demonstration)")

# Load raw data
raw_data_path = config['data']['raw_path']
df_raw = pd.read_csv(raw_data_path)

# For demonstration, we'll use a sample of the data
# In production, this would be your new data
new_data = df_raw.sample(n=min(100, len(df_raw)), random_state=42).copy()

print(f"\n✅ Data loaded")
print(f"  • Records: {len(new_data)}")
print(f"  • Features: {len(new_data.columns)}")

print(f"\n📋 Data Preview:")
safe_display(new_data.head())

In [0]:
# MAGIC %md
# MAGIC ## 7. Data Validation

In [0]:
print("="*60)
print("VALIDATING NEW DATA")
print("="*60)

# Check for required columns
target_col = config['preprocessing']['target']
required_features = [col for col in df_raw.columns if col != target_col]

missing_features = [col for col in required_features if col not in new_data.columns]

if missing_features:
    print(f"\n❌ Missing required features:")
    for feature in missing_features:
        print(f"  • {feature}")
    raise ValueError("Missing required features")
else:
    print(f"\n✅ All required features present")

# Check for missing values
missing_counts = new_data[required_features].isnull().sum()
missing_features = missing_counts[missing_counts > 0]

if len(missing_features) > 0:
    print(f"\n⚠️ Missing values detected:")
    for feature, count in missing_features.items():
        print(f"  • {feature}: {count} ({count/len(new_data)*100:.1f}%)")
    
    print(f"\n🔧 Handling missing values...")
    # Simple imputation - in production, use more sophisticated methods
    for col in missing_features.index:
        if new_data[col].dtype in ['int64', 'float64']:
            new_data[col].fillna(new_data[col].median(), inplace=True)
        else:
            new_data[col].fillna(new_data[col].mode()[0], inplace=True)
    
    print(f"✅ Missing values imputed")
else:
    print(f"✅ No missing values")

# Check data types
print(f"\n📊 Data Types:")
print(new_data[required_features].dtypes.value_counts())

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 8. Feature Engineering

In [0]:
print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

# Initialize feature engineer
feature_engineer = FeatureEngineer(config)

# Create features (same as training)
print(f"\n🔧 Creating features...")
new_data_featured = feature_engineer.create_features(new_data)

print(f"✅ Features created")
print(f"  • Original features: {len(new_data.columns)}")
print(f"  • New features: {len(new_data_featured.columns)}")

# Show new features
original_cols = set(new_data.columns)
new_cols = set(new_data_featured.columns) - original_cols

if new_cols:
    print(f"\n📋 Engineered Features:")
    for col in sorted(new_cols):
        print(f"  • {col}")

print("="*60)

In [0]:
print("="*60)
print("PREPROCESSING DATA")
print("="*60)

# Separate features (remove target if present)
if target_col in new_data_featured.columns:
    X_new = new_data_featured.drop(columns=[target_col])
    y_actual = new_data_featured[target_col]
    has_actuals = True
    print(f"\n📊 Target column found - will compare predictions with actuals")
else:
    X_new = new_data_featured.copy()
    y_actual = None
    has_actuals = False
    print(f"\n📊 No target column - generating predictions only")

# Encode categorical features
print(f"\n🔧 Encoding categorical features...")
X_encoded = X_new.copy()

for col, encoder in label_encoders.items():
    if col in X_encoded.columns:
        # Handle unseen categories
        X_encoded[col] = X_encoded[col].apply(
            lambda x: x if x in encoder.classes_ else encoder.classes_[0]
        )
        X_encoded[col] = encoder.transform(X_encoded[col])

print(f"✅ Categorical features encoded")

# Ensure correct feature order
X_encoded = X_encoded[scaler.feature_names_in_]

print(f"\n🔧 Scaling features...")
X_scaled = scaler.transform(X_encoded)
X_scaled_df = pd.DataFrame(X_scaled, columns=scaler.feature_names_in_, index=X_encoded.index)

print(f"✅ Features scaled")
print(f"  • Final feature count: {X_scaled_df.shape[1]}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 10. Generate Predictions

In [0]:
print("="*60)
print("GENERATING PREDICTIONS")
print("="*60)

# Make predictions
print(f"\n🔮 Making predictions...")
predictions = production_model.predict(X_scaled_df)

print(f"✅ Predictions generated")
print(f"  • Total predictions: {len(predictions)}")

# Calculate prediction statistics
pred_stats = {
    'mean': predictions.mean(),
    'median': np.median(predictions),
    'std': predictions.std(),
    'min': predictions.min(),
    'max': predictions.max(),
    'q25': np.percentile(predictions, 25),
    'q75': np.percentile(predictions, 75)
}

print(f"\n📊 Prediction Statistics:")
print(f"  • Mean:   ${pred_stats['mean']:,.2f}")
print(f"  • Median: ${pred_stats['median']:,.2f}")
print(f"  • Std:    ${pred_stats['std']:,.2f}")
print(f"  • Min:    ${pred_stats['min']:,.2f}")
print(f"  • Max:    ${pred_stats['max']:,.2f}")
print(f"  • Q25:    ${pred_stats['q25']:,.2f}")
print(f"  • Q75:    ${pred_stats['q75']:,.2f}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 11. Calculate Prediction Confidence Intervals

In [0]:
print("="*60)
print("CALCULATING CONFIDENCE INTERVALS")
print("="*60)

# For tree-based models, we can estimate uncertainty using prediction variance
# For linear models, we use residual standard error

model_type = type(production_model).__name__

if hasattr(production_model, 'estimators_'):
    # Ensemble model - use prediction variance across trees
    print(f"\n🔧 Using ensemble variance for confidence intervals...")
    
    # Get predictions from all estimators
    all_predictions = np.array([tree.predict(X_scaled_df) for tree in production_model.estimators_])
    
    # Calculate standard deviation across predictions
    prediction_std = all_predictions.std(axis=0)
    
    # 95% confidence interval (±1.96 * std)
    confidence_lower = predictions - 1.96 * prediction_std
    confidence_upper = predictions + 1.96 * prediction_std
    
    print(f"✅ Confidence intervals calculated using ensemble variance")
    
else:
    # Non-ensemble model - use fixed percentage based on model performance
    print(f"\n🔧 Using model error for confidence intervals...")
    
    # Use model's test RMSE as uncertainty estimate
    model_rmse = metrics.get('test_rmse', predictions.std() * 0.2)
    
    # 95% confidence interval (±1.96 * RMSE)
    confidence_lower = predictions - 1.96 * model_rmse
    confidence_upper = predictions + 1.96 * model_rmse
    
    print(f"✅ Confidence intervals calculated using RMSE: ${model_rmse:,.2f}")

# Calculate interval width
interval_width = confidence_upper - confidence_lower

print(f"\n📊 Confidence Interval Statistics:")
print(f"  • Mean Width: ${interval_width.mean():,.2f}")
print(f"  • Median Width: ${np.median(interval_width):,.2f}")
print(f"  • Min Width: ${interval_width.min():,.2f}")
print(f"  • Max Width: ${interval_width.max():,.2f}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 12. Create Results DataFrame

In [0]:
print("="*60)
print("CREATING RESULTS DATAFRAME")
print("="*60)

# Create results dataframe
results_df = new_data.copy()
results_df['Predicted_Price'] = predictions
results_df['Confidence_Lower_95'] = confidence_lower
results_df['Confidence_Upper_95'] = confidence_upper
results_df['Confidence_Interval_Width'] = interval_width
results_df['Prediction_Timestamp'] = datetime.now()
results_df['Model_Version'] = production_version.version

# Add actual vs predicted comparison if actuals available
if has_actuals:
    results_df['Actual_Price'] = y_actual.values
    results_df['Prediction_Error'] = results_df['Actual_Price'] - results_df['Predicted_Price']
    results_df['Absolute_Error'] = np.abs(results_df['Prediction_Error'])
    results_df['Percentage_Error'] = (results_df['Prediction_Error'] / results_df['Actual_Price'] * 100)
    
    print(f"\n✅ Results with actual comparisons created")
    
    # Calculate accuracy metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    rmse = np.sqrt(mean_squared_error(y_actual, predictions))
    mae = mean_absolute_error(y_actual, predictions)
    r2 = r2_score(y_actual, predictions)
    
    print(f"\n📊 Prediction Accuracy:")
    print(f"  • RMSE: ${rmse:,.2f}")
    print(f"  • MAE:  ${mae:,.2f}")
    print(f"  • R²:   {r2:.4f}")
else:
    print(f"\n✅ Results created (predictions only)")

print(f"\n📋 Results Preview:")
display_cols = ['Predicted_Price', 'Confidence_Lower_95', 'Confidence_Upper_95']
if has_actuals:
    display_cols = ['Actual_Price'] + display_cols + ['Prediction_Error', 'Percentage_Error']

safe_display(results_df[display_cols].head(10))

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 13. Visualize Predictions

In [0]:
print("="*60)
print("VISUALIZING PREDICTIONS")
print("="*60)

if has_actuals:
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Actual vs Predicted
    axes[0, 0].scatter(y_actual, predictions, alpha=0.6, s=50)
    min_val = min(y_actual.min(), predictions.min())
    max_val = max(y_actual.max(), predictions.max())
    axes[0, 0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    axes[0, 0].set_xlabel('Actual Price ($)', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Predicted Price ($)', fontsize=12, fontweight='bold')
    axes[0, 0].set_title(f'Actual vs Predicted (R² = {r2:.4f})', fontsize=13, fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Prediction errors
    axes[0, 1].hist(results_df['Prediction_Error'], bins=30, edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(x=0, color='r', linestyle='--', lw=2)
    axes[0, 1].set_xlabel('Prediction Error ($)', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[0, 1].set_title(f'Prediction Error Distribution (MAE = ${mae:,.2f})', fontsize=13, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. Predictions with confidence intervals
    sorted_idx = np.argsort(predictions)
    axes[1, 0].plot(range(len(predictions)), predictions[sorted_idx], 'b-', label='Prediction', linewidth=2)
    axes[1, 0].fill_between(range(len(predictions)), 
                            confidence_lower[sorted_idx], 
                            confidence_upper[sorted_idx], 
                            alpha=0.3, label='95% Confidence Interval')
    axes[1, 0].scatter(range(len(predictions)), y_actual.values[sorted_idx], 
                      alpha=0.5, s=20, color='red', label='Actual', zorder=5)
    axes[1, 0].set_xlabel('Sample (sorted by prediction)', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Price ($)', fontsize=12, fontweight='bold')
    axes[1, 0].set_title('Predictions with Confidence Intervals', fontsize=13, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Percentage error distribution
    axes[1, 1].hist(results_df['Percentage_Error'], bins=30, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 1].axvline(x=0, color='r', linestyle='--', lw=2)
    axes[1, 1].set_xlabel('Percentage Error (%)', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[1, 1].set_title('Percentage Error Distribution', fontsize=13, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
else:
    # Predictions only visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # 1. Prediction distribution
    axes[0].hist(predictions, bins=30, edgecolor='black', alpha=0.7)
    axes[0].axvline(x=predictions.mean(), color='r', linestyle='--', lw=2, label=f'Mean: ${predictions.mean():,.2f}')
    axes[0].set_xlabel('Predicted Price ($)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[0].set_title('Prediction Distribution', fontsize=13, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 2. Predictions with confidence intervals
    sorted_idx = np.argsort(predictions)
    axes[1].plot(range(len(predictions)), predictions[sorted_idx], 'b-', label='Prediction', linewidth=2)
    axes[1].fill_between(range(len(predictions)), 
                        confidence_lower[sorted_idx], 
                        confidence_upper[sorted_idx], 
                        alpha=0.3, label='95% Confidence Interval')
    axes[1].set_xlabel('Sample (sorted by prediction)', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Price ($)', fontsize=12, fontweight='bold')
    axes[1].set_title('Predictions with Confidence Intervals', fontsize=13, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Visualizations complete")
print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 14. Save Predictions

In [0]:
print("="*60)
print("SAVING PREDICTIONS")
print("="*60)

# Save predictions
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
predictions_path = f"{processed_path}batch_predictions_{timestamp}.csv"

results_df.to_csv(predictions_path, index=False)

print(f"\n✅ Predictions saved")
print(f"  • File: {predictions_path}")
print(f"  • Records: {len(results_df)}")

# Create summary report
summary_report = {
    'timestamp': datetime.now().isoformat(),
    'model_name': model_registry_name,
    'model_version': production_version.version,
    'model_type': model_type,
    'total_predictions': len(predictions),
    
    'prediction_statistics': {
        'mean': float(pred_stats['mean']),
        'median': float(pred_stats['median']),
        'std': float(pred_stats['std']),
        'min': float(pred_stats['min']),
        'max': float(pred_stats['max'])
    },
    
    'confidence_intervals': {
        'mean_width': float(interval_width.mean()),
        'median_width': float(np.median(interval_width))
    }
}

if has_actuals:
    summary_report['accuracy_metrics'] = {
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

# Save summary
summary_path = f"{processed_path}batch_predictions_summary_{timestamp}.json"
with open(summary_path, 'w') as f:
    json.dump(summary_report, f, indent=2)

print(f"✅ Summary report saved")
print(f"  • File: {summary_path}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 15. Log to MLflow

In [0]:
print("="*60)
print("LOGGING TO MLFLOW")
print("="*60)

with mlflow.start_run(run_name=f"batch_predictions_{timestamp}"):
    
    # Log parameters
    mlflow.log_param("model_version", production_version.version)
    mlflow.log_param("num_predictions", len(predictions))
    mlflow.log_param("has_actuals", has_actuals)
    
    # Log prediction statistics
    mlflow.log_metrics({
        "pred_mean": pred_stats['mean'],
        "pred_median": pred_stats['median'],
        "pred_std": pred_stats['std'],
        "pred_min": pred_stats['min'],
        "pred_max": pred_stats['max']
    })
    
    # Log confidence interval statistics
    mlflow.log_metrics({
        "ci_mean_width": interval_width.mean(),
        "ci_median_width": np.median(interval_width)
    })
    
    # Log accuracy metrics if available
    if has_actuals:
        mlflow.log_metrics({
            "batch_rmse": rmse,
            "batch_mae": mae,
            "batch_r2": r2
        })
    
    # Log artifacts
    mlflow.log_artifact(predictions_path)
    mlflow.log_artifact(summary_path)
    
    print(f"\n✅ Logged to MLflow")
    print(f"  • Run name: batch_predictions_{timestamp}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 16. Batch Prediction Summary

In [0]:
print("="*60)
print("BATCH PREDICTION SUMMARY")
print("="*60)

print(f"\n📅 Execution Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n🤖 Model Information:")
print(f"  • Model: {model_registry_name}")
print(f"  • Version: {production_version.version}")
print(f"  • Type: {model_type}")

print(f"\n📊 Predictions:")
print(f"  • Total: {len(predictions)}")
print(f"  • Mean: ${pred_stats['mean']:,.2f}")
print(f"  • Range: ${pred_stats['min']:,.2f} - ${pred_stats['max']:,.2f}")

print(f"\n📏 Confidence Intervals:")
print(f"  • Mean Width: ${interval_width.mean():,.2f}")
print(f"  • Coverage: 95%")

if has_actuals:
    print(f"\n🎯 Accuracy:")
    print(f"  • RMSE: ${rmse:,.2f}")
    print(f"  • MAE:  ${mae:,.2f}")
    print(f"  • R²:   {r2:.4f}")

print(f"\n📁 Generated Files:")
print(f"  • batch_predictions_{timestamp}.csv")
print(f"  • batch_predictions_summary_{timestamp}.json")

print(f"\n💡 Next Steps:")
print(f"  • Review predictions in saved CSV file")
print(f"  • Share results with stakeholders")
print(f"  • Monitor prediction quality over time")
if has_actuals:
    print(f"  • Compare with monitoring thresholds")
    print(f"  • Trigger retraining if accuracy degrades")

print(f"\n✅ Batch prediction pipeline complete!")
print("="*60)